In [24]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import json
import pathlib
import matplotlib.pyplot as plt
import numpy as np

from lieroy import se3

from recov.datasets import KittiDataset, EthzDataset
from recova.learning.learning import model_from_file
from recova.registration_result_database import RegistrationPairDatabase

from recova.recov_plot import plot_trajectory_translation, plot_covariance
from recova.trajectory_evaluation import collect_trajectory_data, make_cumulative_covariances 
from recova.util import set_matplotlibrc

np.set_printoptions(precision=3, linewidth=100)

ImportError: cannot import name 'plot_trajectory_translation'

In [26]:
DATABASE_LOCATION = '/home/dlandry/dataset/db_kitti_02/'
LOCATION = '03'
POINTCLOUD_DATASET_LOCATION = '/home/dlandry/dataset/kitti/dataset/sequences/{}'.format(LOCATION)
LEARNING_DATASET_LOCATION = '/home/dlandry/dataset/learning_sets/2018-08-28-kitti-3-4-5.json'
MODEL_LOCATION = '/home/dlandry/dataset/learning_runs/2018-08-29-kitti-03.model'


db = RegistrationPairDatabase(DATABASE_LOCATION)
dataset = KittiDataset(pathlib.Path(POINTCLOUD_DATASET_LOCATION))

with open(LEARNING_DATASET_LOCATION) as f:
    learning_dataset = json.load(f)
    
model = model_from_file(MODEL_LOCATION, 'cello')

In [27]:
DATABASE_LOCATION = '/home/dlandry/dataset/db_eth_05//'
LOCATION = 'wood_autumn'
POINTCLOUD_DATASET_LOCATION = '/home/dlandry//dataset/ethz/{}'.format(LOCATION)
LEARNING_DATASET_LOCATION = '/home/dlandry/dataset/learning_sets/2018-08-20-wood.json'
MODEL_LOCATION = '/home/dlandry/dataset/learning_runs/2018-08-20-wood-summer.model'


db = RegistrationPairDatabase(DATABASE_LOCATION)
dataset = EthzDataset(pathlib.Path(POINTCLOUD_DATASET_LOCATION))

with open(LEARNING_DATASET_LOCATION) as f:
    learning_dataset = json.load(f)
    
model = model_from_file(MODEL_LOCATION, 'cello')

In [ ]:
(gt_trajectory, 
 sampled_trajectory,
 cum_covariances, 
 censi_cum_cov) = collect_trajectory_data(db, LOCATION, dataset, learning_dataset['metadata']['descriptor_config'], model, n_sampled_trajectories=10)

0
32
  0%|                                                    | 0/31 [00:00<?, ?it/s](5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
 26%|███████████▎                                | 8/31 [00:01<00:03,  6.77it/s](5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
 52%|██████████████████████▏                    | 16/31 [00:01<00:01,  8.57it/s](5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
 77%|█████████████████████████████████▎         | 24/31 [00:02<00:00,  8.60it/s](5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
(5000, 6)
100%|█████████████████████████████████████████| 31/31 [00:00<00:00, 1096.82it/s]


In [ ]:
fig, ax = plt.subplots()
ax.grid(color='0.8', linewidth=0.5, zorder=0, linestyle='--')

plot_trajectory_translation(gt_trajectory[:], ax)

for t in sampled_trajectory:
    plot_trajectory_translation(t[0:], ax, linestyle='dotted')    

for i in range(0, len(gt_trajectory)):
    plot_covariance(gt_trajectory[i], cum_covariances[i], ax, color='0.6')
    
ax.set_xlabel('$x$ (m)')
ax.set_ylabel('$y$ (m)')
    
plt.show()

In [ ]:
cum_covariances